# Pricing and Risk Calculation - Contexts
In this tutorial we calculate pricing and risk in different pricing environments

In [1]:
import datetime as dt
from gs_quant.common import PayReceive
from gs_quant.instrument import IRSwap, IRSwaption
from gs_quant.markets import PricingContext
from gs_quant.session import Environment, GsSession
from gs_quant import risk

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

### PricingContext

The current `PricingContext` shows current pricing environment and market data paramaterization

In [ ]:
print(PricingContext.current.market.location)
print(PricingContext.current.market.date)
print(PricingContext.current.pricing_date)

We can leverage the `PricingContext` to modify pricing environment and market data

In [ ]:
# Calculating Risk as of a specific date and market location
custom_date = dt.date(2019, 9, 20)
swap = IRSwap(PayReceive.Receive, '10y', 'GBP', fixed_rate='atm+50', notional_amount=1e8)
with PricingContext(pricing_date=custom_date, market_data_location='NYC'):
    delta = swap.calc(risk.IRDeltaParallel)

print(delta.result())

### HistoricalPricingContext
Calculate pricing and risk over a range of dates

In [ ]:
# Short Dated Swaption Historical Pricing - 2Q19
from gs_quant.markets import HistoricalPricingContext
from datetime import date

start_date = date(2019, 4, 1)
end_date = date(2019, 6, 30)

swaption = IRSwaption(PayReceive.Receive, '5y', 'USD', expiration_date='3m', notional_amount=1e8)
with PricingContext(pricing_date=start_date):
    swaption.resolve()  # Freeze dates/strike in contract as of start date

with HistoricalPricingContext(start_date, end_date):
    swaption_res = swaption.calc((risk.DollarPrice, risk.IRGammaParallel,))

prices = swaption_res[risk.DollarPrice]
gamma = swaption_res[risk.IRGammaParallel]

In [ ]:
prices.plot(title='3m5y USD Swaption Price by Date', rot=45)

In [ ]:
gamma.plot(title='3m5y USD Swaption Gamma by Date, Fixed Date & Strike', rot=45)